In [5]:
import pandas as pd
file_path = '/home/jovyan/git/rsm-mgta453-assignment1-rsm-yih070/data/prices.parquet'
prices_data = pd.read_parquet(file_path)

In [7]:
def optimize_prices(df):
    price_summary = df.groupby('price').agg({
        'mon_purchases': 'sum',
        'tues_purchases': 'sum'
    }).reset_index()
    
    price_summary['tues_revenue'] = price_summary['price'] * price_summary['tues_purchases']
    
    tues_opt_price = price_summary.loc[price_summary['tues_revenue'].idxmax(), 'price']
    
    def adjusted_monday_revenue(price, prices_data, optimal_price_tuesday):
        monday_data = prices_data[prices_data['price'] == price]
        
        monday_purchases = monday_data['mon_purchases'].sum()
        potential_tuesday_purchases = monday_data['mon_purchases'].count() - monday_purchases
        
        monday_revenue = price * monday_purchases
        additional_tuesday_revenue = optimal_price_tuesday * potential_tuesday_purchases

        return monday_revenue + additional_tuesday_revenue
    
    price_summary['adjusted_mon_revenue'] = price_summary['price'].apply(
        lambda x: adjusted_monday_revenue(x, df, tues_opt_price)
    )
    
    mon_opt_price = price_summary.loc[price_summary['adjusted_mon_revenue'].idxmax(), 'price']
    
    return mon_opt_price, tues_opt_price



(60, 32)